In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Scattering

Let us consider a classical theory of scattering, specifically we have a potential fixed in space that falls off far from the origin and we imagine a large number of particles incident upon this from a large distance.  By examining the trajectories of the particles well after the scattering "event" we would try to reconstruct what happened.  Rather than use our "scattering formalism", developed in class, let us consider this to be a problem involving integrating the equations of motion for many particles in a potential.

To make our lives just a little easier, let us assume $m_1/m_2\to 0$, so that the lab frame and the COM frame are coincident and we can treat the scattering potential as "fixed" in space.  We will also work throughout in the $z=0$ plane, so this is effectively a 2D problem.  Both of these simplifications can be relaxed if needed.

Let us consider the trial potential
$$
  V = \frac{1}{1+r^2} \quad\mathrm{with}\quad r^2=(x^2+y^2)
$$
in some units in polar coordinates, $(r,\theta)$ or Cartesian coordinates, $(x,y)$.

We take a "beam" of $N_p=128$ equally spaced, unit mass particles running vertifcally from $y=1$ to $y=8$ at $x=-20$ with $\vec{v}_0=(1,0)$.  Note the potential is symmetric in $y\to -y$ so we don't need to directly include particles with $y<0$.  If we didn't know that, we could obviously simulate those too.

We are interested in the distribution of particles well after the scattering, i.e. as $x\to\infty$.

Our equations of motion are straightforward (recall $m=1$):
$$
  \dot{\vec{x}} = \vec{p} \quad , \quad
  \dot{\vec{p}} = -\nabla V(\vec{x}) = -\frac{dV}{dr^2}\nabla r^2
$$
with the initial conditions given above.

**Note** Rather than simulating each particle individually, we can evolve them all simultaneously.  They don't interact at all, they're all feeling the same "external" potential $V$.

In [ ]:
# Set up some variables for our problem -- start with the initial
# conditions.
# We have Np particles.  We store the dynamical variables
# as x for particle 1, followed by x for particle 2, ...
# followed by y for particle 1, y for particle 2, ...
# followed by vx for particle 1, followed by ...
# followed by vy for particle 1, followed by ...
Np = 128            # The number of particles to follow.
y0 = np.zeros(4*Np) # Positions and velocities for each dimension and each particle.
y0[0*Np:1*Np] = -20.0                       # Initial x-position.
y0[1*Np:2*Np] = np.linspace(1.0,8.0,Np)     # Initial y-position.
y0[2*Np:3*Np] = 1.0                         # Initial vx.
# and the initial y velocity is zero.

In [ ]:
def derivs(t,yy):
    """Returns dy given yy."""
    # Make space for the derivatives, and set the problem size.
    NN = yy.size//4
    dy = np.zeros_like(yy)
    # Do a little unpacking to make the code easier to read.
    xpos = yy[0*NN:1*NN]
    ypos = yy[1*NN:2*NN] 
    r2   = xpos**2 + ypos**2 
    V    = 1.0/(1.0+r2)
    dVdr2= -1/(1+r2)**2
    # FILL IN THE DERIVATIVES
    dy[0*NN:2*NN] = ...
    dy[2*NN:3*NN] = ...
    dy[3*NN:4*NN] = ...
    return(dy)
    #

In [ ]:
from scipy.integrate import solve_ivp
# Use high-accuracy settings in the integration.  This isn't
# really necessary here, but it serves as an example of how
# to do it if you ever need to.
res = solve_ivp(derivs,[0,75],y0,t_eval=np.linspace(0,75,150),rtol=1e-5,method='DOP853')
print(res)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,5))
#
ax.plot([0],[0],'ko',ms=10) # Plot a "scattering center"
for ipart in range(Np):
    xpos = res.y[0*Np+ipart,:]
    ypos = res.y[1*Np+ipart,:]
    ax.plot(xpos,ypos,'-',color=str(0.9*float(ipart)/Np),\
            alpha=0.5,rasterized=True)
#
ax.set_xlim(-10,10)
ax.set_ylim(-1 ,10)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')

In [ ]:
# Figure out the y-positions at x=10 -- some particles may not
# have made it to x=10, if so we discard them (this turns out
# not to happen in this case, but to keep things general...).
ykeep = []
for ipart in range(Np):
    xpos = res.y[0*Np+ipart,:]
    ypos = res.y[1*Np+ipart,:]
    if np.max(xpos)>10.0:
        ykeep.append(np.abs(np.interp(10.0,xpos,ypos)))
# Plot a histogram.
fig,ax = plt.subplots(1,1,figsize=(10,5))
_ = ax.hist(ykeep,bins=np.linspace(4,8,41))
#
ax.set_xlabel(r'$y(x=10)$')
ax.set_ylabel('Number')

In [ ]:
# We can also compute the scattering angle and plot a histogram of that.
xvel = res.y[2*Np:3*Np,-1]
yvel = res.y[3*Np:4*Np,-1]
theta= np.arctan(yvel/xvel)
#
# Plot a histogram.
fig,ax = plt.subplots(1,1,figsize=(10,5))
_ = ax.hist(np.abs(theta),bins=np.linspace(0,np.pi/2,41))
#
ax.set_xlim(0,np.pi/2)
ax.set_xlabel(r'$|\theta|$ (radians)')
ax.set_ylabel('Number')

Compare the deflection to those we would obtain using the Born approximation for small-angle scattering.  You may do any required integrals numerically (the ```scipy.integrate``` routine ```quad``` is a reasonable choice).  At what angle, $\theta_{\rm true}$, does the error first rise to be 10\%?

In [ ]:
from scipy.integrate import quad
#
def theta_born(b):
    """Computes theta for impact parameter b within the Born approximation."""
    FILL THIS IN
    return(tt)

In [ ]:
theta_approx = [theta_born(b) for b in res.y[1*Np:2*Np,0]]
#
fig,ax = plt.subplots(1,2,figsize=(10,4))
# First plot true against approximate:
ax[0].scatter(theta,theta_approx)
ax[0].plot([0,1],[0,1],'k:') # The truth.
ax[0].set_aspect('equal')
ax[0].set_xlim(0,1)
ax[0].set_ylim(0,1)
ax[0].set_xlabel(r'$\theta_{\rm true}$')
ax[0].set_ylabel(r'$\theta_{\rm Born}$')
# then plot the ratio against true:
ax[1].scatter(theta,theta_approx/theta)
ax[1].set_xlim(0,1)
ax[1].set_ylim(1.0,1.2)
ax[1].set_xlabel(r'$\theta_{\rm true}$')
ax[1].set_ylabel(r'$\theta_{\rm Born} / \theta_{\rm true}$')

We see that the Born approximation is not fantastically accurate even when $\theta$ is "small".  We could gain more accuracy by going to higher order in our expansion or by solving for the deflection "properly" using our $V_{\rm eff}$ formalism and the integrals derived in class.  But not today!

***Note*** that computing the scattering cross section is now relatively straightforward -- we just need a scheme for computing the derivative of the scattering angle with respect to the impact parameter, which could be done via a spline fit, or finite differencing, or by differentating our integral expression for $\theta(b)$ to get a new integral we do numerically.

# The End